# Tarefa Aula 7 Engenharia de Dados Awari
## Rogerio Veiga Andrade - 10/11/2023

### Importar as diferenças que são criadas nos dados da pasta /exercicios/municipios-estados/streaming/. A cada vez que um novo arquivo for adicionado, o mesmo deve ser importado para a pasta da UF correspondente e adicionado ao fim do arquivo cidades.csv.
### Utilizar Apache Kafka para fazer o mesmo processo, mas de maneira automatizada.

In [ ]:
import pyspark
import boto3
import pandas as pd

from io import StringIO
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
from pyspark.sql.types import ArrayType, DoubleType, BooleanType
from pyspark.sql.functions import col,array_contains

In [ ]:
# Criando o objeto client

client = boto3.client('s3',
    endpoint_url='http://awari-minio-nginx:9000',
    aws_access_key_id='6DdjRBILhyMi2G2i',
    aws_secret_access_key='9LSWJZT7f0i8FgjzflQQYrOq9tvUQrD1',
    aws_session_token=None,
    config=boto3.session.Config(signature_version='s3v4'),
    verify=False,
    region_name='sa-east-1'
)

In [ ]:
estados = pd.read_csv('arquivos/streaming/estados.csv')
estados.head()

,codigo_uf,uf,nome,latitude,longitude,regiao
0,101,AWARI-1,Awari Estado 1,-10.83,-63.34,Norte
1,102,AWARI-2,Awari Estado 2,-8.77,-70.55,Norte


In [ ]:
for uf in estados['uf'].unique():
    client.put_object(Bucket='aula-06', Key=f'{uf}/')

# Lista dos prefixos (pastas) no bucket

response = client.list_objects_v2(Bucket='aula-06', Delimiter='/')
response['CommonPrefixes']

[{'Prefix': 'AC/'},
 {'Prefix': 'AL/'},
 {'Prefix': 'AM/'},
 {'Prefix': 'AP/'},
 {'Prefix': 'AWARI-1/'},
 {'Prefix': 'AWARI-2/'},
 {'Prefix': 'BA/'},
 {'Prefix': 'CE/'},
 {'Prefix': 'DF/'},
 {'Prefix': 'ES/'},
 {'Prefix': 'GO/'},
 {'Prefix': 'MA/'},
 {'Prefix': 'MG/'},
 {'Prefix': 'MS/'},
 {'Prefix': 'MT/'},
 {'Prefix': 'PA/'},
 {'Prefix': 'PB/'},
 {'Prefix': 'PE/'},
 {'Prefix': 'PI/'},
 {'Prefix': 'PR/'},
 {'Prefix': 'RJ/'},
 {'Prefix': 'RN/'},
 {'Prefix': 'RO/'},
 {'Prefix': 'RR/'},
 {'Prefix': 'RS/'},
 {'Prefix': 'SC/'},
 {'Prefix': 'SE/'},
 {'Prefix': 'SP/'},
 {'Prefix': 'TO/'},
 {'Prefix': 'exemplos/'}]

In [ ]:
municipios = pd.read_csv('arquivos/streaming/municipios.csv')
municipios.head()

,codigo_ibge,nome,latitude,longitude,capital,codigo_uf,siafi_id,ddd,fuso_horario
0,9900001,Minha cidade 1,-16.7573,-49.4412,0,52,1050,100,America/Sao_Paulo
1,9900002,Minha cidade 2,-18.4831,-47.3916,0,31,4001,101,America/Sao_Paulo


In [ ]:
estados = pd.read_json('arquivos/estados.json')
municipios_estados = municipios.merge(estados, on='codigo_uf', how='inner')
municipios_estados.head()

,codigo_ibge,nome_x,latitude_x,longitude_x,capital,codigo_uf,siafi_id,ddd,fuso_horario,uf,nome_y,latitude_y,longitude_y,regiao
0,9900001,Minha cidade 1,-16.7573,-49.4412,0,52,1050,100,America/Sao_Paulo,GO,Goiás,-15.98,-49.86,Centro-Oeste
1,9900002,Minha cidade 2,-18.4831,-47.3916,0,31,4001,101,America/Sao_Paulo,MG,Minas Gerais,-18.10,-44.38,Sudeste


In [ ]:
# Para cada estado presente na lista de municipios, o código lê o arquivo de cidades do estado corresponde, adiciona os municipios no DataFrame, converte pra csv e devolve pro MinIO

for uf in municipios_estados['uf'].unique():
    df = pd.read_csv(client.get_object(Bucket='aula-06', Key=f'{uf}/cidades.csv')['Body'])
    df = pd.concat([ df, municipios_estados[municipios_estados['uf'] == uf] ])
    csv_buffer = StringIO()
    df.to_csv(csv_buffer)
    client.put_object(Body=csv_buffer.getvalue(), Bucket='aula-06', Key=f'{uf}/cidades.csv')

# KAFKA

In [1]:
!wget https://downloads.apache.org/kafka/3.6.0/kafka_2.13-3.6.0.tgz

!tar xvzf kafka_2.13-3.6.0.tgz

# inicializa o zookeeper que é uma ferramenta de configuração utilizada pelo kafka
!kafka_2.13-3.6.0/bin/zookeeper-server-start.sh -daemon kafka_2.13-3.6.0/config/zookeeper.properties

# inicializa o kafka
!kafka_2.13-3.6.0/bin/kafka-server-start.sh -daemon kafka_2.13-3.6.0/config/server.properties

# cria o topico, que é o canal onde os dados vão ser enviados e consumidos
!kafka_2.13-3.6.0/bin/kafka-topics.sh --create --bootstrap-server 127.0.0.1:9092 --topic awari

# --describe lista as caracteristicas do topico
!kafka_2.13-3.6.0/bin/kafka-topics.sh --describe --bootstrap-server 127.0.0.1:9092 --topic awari

# --describe lista as caracteristicas do topico
!kafka_2.13-3.6.0/bin/kafka-topics.sh --describe --bootstrap-server 127.0.0.1:9092 --topic awari



--2023-11-10 16:39:06--  https://downloads.apache.org/kafka/3.6.0/kafka_2.13-3.6.0.tgz
Resolving downloads.apache.org (downloads.apache.org)... 88.99.95.219, 135.181.214.104, 2a01:4f9:3a:2c57::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|88.99.95.219|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 113257079 (108M) [application/x-gzip]
Saving to: ‘kafka_2.13-3.6.0.tgz’

kafka_2.13-3.6.0.tg 100%[===================>] 108.01M  23.9MB/s    in 5.4s    

2023-11-10 16:39:12 (19.8 MB/s) - ‘kafka_2.13-3.6.0.tgz’ saved [113257079/113257079]

kafka_2.13-3.6.0/
kafka_2.13-3.6.0/LICENSE
kafka_2.13-3.6.0/NOTICE
kafka_2.13-3.6.0/bin/
kafka_2.13-3.6.0/bin/kafka-delete-records.sh
kafka_2.13-3.6.0/bin/trogdor.sh
kafka_2.13-3.6.0/bin/kafka-jmx.sh
kafka_2.13-3.6.0/bin/connect-mirror-maker.sh
kafka_2.13-3.6.0/bin/kafka-console-consumer.sh
kafka_2.13-3.6.0/bin/kafka-consumer-perf-test.sh
kafka_2.13-3.6.0/bin/kafka-log-dirs.sh
kafka_2.13-3.6.0/bin/kafka-metadata

In [2]:
!pip install kafka-python
from kafka import KafkaProducer, KafkaConsumer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 4.3 MB/s eta 0:00:00


In [26]:
kafka_producer = KafkaProducer(bootstrap_servers='127.0.0.1:9092')

def envia_mensagem(producer, topic_name, key, valor):
    key = bytes(key, encoding='utf-8')
    valor = bytes(valor, encoding='utf-8')
    producer.send(topic_name, key = key, value=valor)
    producer.flush()

In [10]:
import pandas as pd
municipios_csv = pd.read_csv('./municipios.csv')
estados = pd.read_json('./estados.json')
municipios_estados = municipios_csv.merge(estados, on='codigo_uf', how='inner')
municipios_estados.head()



,codigo_ibge,nome_x,latitude_x,longitude_x,capital,codigo_uf,siafi_id,ddd,fuso_horario,uf,nome_y,latitude_y,longitude_y,regiao
0,9900001,Minha cidade 1,-16.7573,-49.4412,0,52,1050,100,America/Sao_Paulo,GO,Goiás,-15.98,-49.86,Centro-Oeste
1,9900002,Minha cidade 2,-18.4831,-47.3916,0,31,4001,101,America/Sao_Paulo,MG,Minas Gerais,-18.10,-44.38,Sudeste


In [27]:

mensagens = []
for index, row in municipios_estados.iterrows():
    mensagens.append({'ticker': row['uf'], 'valor': row['nome_x']})

print(mensagens)

[{'ticker': 'GO', 'valor': 'Minha cidade 1'}, {'ticker': 'MG', 'valor': 'Minha cidade 2'}]


In [28]:
for mensagem in mensagens:
    envia_mensagem(
        kafka_producer,
        'awari',
        mensagem['ticker'],
        str(mensagem['valor'])
    )

kafka_producer.close()

In [29]:
consumer = KafkaConsumer('awari', bootstrap_servers='127.0.0.1:9092', consumer_timeout_ms=1000, auto_offset_reset='earliest')

In [30]:
for msg in consumer:
    print(msg.key)
    print(msg.value)
    df_tmp = pd.DataFrame({'nome': [msg.value], 'uf': msg.key})
    df = pd.read_csv(client.get_object(Bucket='aula-06', Key=f'{msg.key}/cidades.csv')['Body'])
    df = pd.concat([ df, df_tmp ])
    csv_buffer = StringIO()
    df.to_csv(csv_buffer)
    client.put_object(Body=csv_buffer.getvalue(), Bucket='aula-06', Key=f'{msg.key}/cidades.csv')
consumer.close()

b'GO'
b'Minha cidade 1'
b'MG'
b'Minha cidade 2'
b'GO'
b'Minha cidade 1'
b'MG'
b'Minha cidade 2'


ERROR:kafka.consumer.fetcher:Fetch to node 0 failed: Cancelled: <BrokerConnection node_id=0 host=f1d7f9a89ead:9092 <connected> [IPv4 ('172.28.0.12', 9092)]>
